# ch2-使用大语言模型

## 1.本节实战从4个方面演示使用大语言模型（LLM）的能力

- <font size=3>阿里开源 `modelscope` 调用本地开源模型 `qwen2`</font>
- <font size=3>huggingface `transformers` 调用本地开源模型 `chatglm3`</font>
- <font size=3>通过Http API在线调用讯飞星火spark大模型</font>
- <font size=3>通过Http API 调用ollama部署`qwen2`模型</font>

### 还能学习
- <font size=3>查看GPU</font>
- <font size=3>流式输出</font>

## 2.查看GPU

- <font size=3> `nvidia-smi` 查看机器上的nvidia显卡 </font>
   - 驱动
   - cuda
   - 显卡型号
   - 显存
   - 利用率

In [24]:
! nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fri Oct  4 16:04:01 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 450.57       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:04:00.0 Off |                  N/A |
| 40%   28C    P8     1W / 280W |  13896MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:08:00.0 Off |                  N/A |
| 41%   

- 设置程序可见的显卡
   - `os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'` 只使用第0和第1个显卡 

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

## 3. 控制大语言模型的输出的随机性的参数

大语言模型预测下一个token时会先输出所有token的概率值，有不同的方法来控制选择哪一个token作为输出，主要以下4个参数

- 温度（Temperature）: 起到平滑调整概率的作用，temperature=1时，原始概率保持不变，temperature<1时，原来概率大的会变得更大（概率集中效果），temperature>1时,概率值越平均

- Top-K: 模型输出是在概率在top-k的范围里随机选择一个，K值越大，选择范围越广，生成的文本越多样；K值越小，选择范围越窄，生成的文本越趋向于高概率的词。 k=1就是直接选择最高概率的token输出

- Top-p: 通过累积概率来限定范围，top-p=0.5表示随机采样的范围是概率在前50%的tokens， top-p选择的tokens数是动态的

- max-tokens: max-tokens参数指定了模型在停止生成之前可以生成的最大token（或词）数量


### $$
[
\text{Softmax}_T(z)_i = \frac{e^{\frac{z_i}{T}}}{\sum_{j} e^{\frac{z_j}{T}}}
]
$$

In [3]:
import torch
import torch.nn.functional as F
import numpy as np

In [4]:
inputs = np.array([[2.0, 1.0, 0.1]])

In [5]:
temperature = 1
logits = torch.tensor(inputs / temperature) 
softmax_scores = F.softmax(logits, dim=1)
print(f"temperature = {temperature} {softmax_scores.cpu().numpy()}")

temperature = 1 [[0.65900114 0.24243297 0.09856589]]


In [6]:
temperature = 0.1
logits = torch.tensor(inputs / temperature) 
softmax_scores = F.softmax(logits, dim=1)
print(f"temperature = {temperature} {softmax_scores.cpu().numpy()}")

temperature = 0.1 [[9.99954597e-01 4.53978684e-05 5.60254205e-09]]


In [7]:
temperature = 10
logits = torch.tensor(inputs / temperature) 
softmax_scores = F.softmax(logits, dim=1)
print(f"temperature = {temperature} {softmax_scores.cpu().numpy()}")

temperature = 10 [[0.36605947 0.33122431 0.30271622]]


## 4. `modelscope` 调用本地开源模型 `qwen2`

#### Qwen2是由阿里云通义千问团队研发的新一代大型语言模型系列

Qwen2系列提供了多个不同规模的模型，以满足不同场景和计算资源的需求，具体包括：

* Qwen2-0.5B
* Qwen2-1.5B
* Qwen2-7B
* Qwen2-57B-A14B（混合专家模型，MoE）
* Qwen2-72B

这些模型在参数数量上从数亿到数百亿不等，为用户提供了丰富的选择。


* 相比前代模型Qwen1.5，Qwen2在代码、数学、推理、指令遵循、多语言理解等多个方面实现了性能的显著提升。
* 特别是在超长上下文处理方面，Qwen2-72B-Instruct模型支持处理长达**128K tokens**的上下文，这在大型文档理解和复杂对话处理中尤为重要。

在原有的中文和英文基础上，Qwen2新增了27种语言的高质量数据，使得模型在多语言处理上更加出色。

Qwen2模型在**ModelScope**和**Hugging Face**平台上可以在线体验


#### 下载
- https://www.modelscope.cn/models/qwen/Qwen2-7B-Instruct/files

``` shell
git clone https://www.modelscope.cn/qwen/Qwen2-7B-Instruct.git
```
#### 安装

``` shell
# modelscope==1.16.1
pip install modelscope

# pip install optimum
# pip install auto-gptq
# import autogptq_cuda_64

```


### 调用

In [8]:
from modelscope import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [9]:
model_path = './data/llm_app/llm/Qwen2-7B-Instruct/'

model = AutoModelForCausalLM.from_pretrained(model_path,
                                            device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)
gen_config = GenerationConfig.from_pretrained(model_path)


/root/anaconda3/envs/llm/lib/python3.9/site-packages/accelerate/utils/modeling.py:1365: UserWarning: Current model requires 704648448 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [04:49<00:00, 72.36s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
tokenizer("你好, are you ok")

{'input_ids': [108386, 11, 525, 498, 5394], 'attention_mask': [1, 1, 1, 1, 1]}

In [14]:
tokenizer.decode(108386)

'你好'

In [15]:
tokenizer.decode(11)

','

In [16]:
gen_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "repetition_penalty": 1.05,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}

In [22]:
def run_prompt(prompt, temperature=0.1, top_k=20, top_p=0.8, max_new_tokens=2048):
    gen_config.temperature = temperature
    gen_config.top_k = top_k
    gen_config.top_p = top_p
    
    messages = [
        {"role": "system", "content": ""},
        {"role": "user", "content": prompt}
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    print(text)
    
    model_input = tokenizer([text], return_tensors='pt').to('cuda')
    
    generated_ids = model.generate(model_input.input_ids,
                                  max_new_tokens=max_new_tokens,
                                  generation_config=gen_config)
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_input.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response

In [23]:
prompt = 'hello'
run_prompt(prompt)

<|im_start|>system
<|im_end|>
<|im_start|>user
hello<|im_end|>
<|im_start|>assistant



'Hello! How can I assist you today?'

## 5. huggingface `transformers` 调用本地开源模型 `chatglm3`

### ChatGLM全名General Language Model，是智谱AI自主研发的大型语言模型
- 开源的产品 chatglm3-6B GLM-9B

chatglm3模型在**ModelScope**和**Hugging Face**平台上可以在线体验


#### 下载
- https://www.modelscope.cn/models/ZhipuAI/chatglm3-6b-32k/files

``` shell
git clone https://www.modelscope.cn/ZhipuAI/chatglm3-6b-32k.git
```
#### 安装

``` shell
# transformers==4.41.2
pip install transformers

```

### 调用chatglm3

In [1]:
from transformers import AutoModel, AutoTokenizer

In [2]:
model_path = './data/llm_app/llm/chatglm3-6b-32k/'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_path,
                                         trust_remote_code=True)

model = AutoModel.from_pretrained(model_path,
                                 device_map="auto",
                                 trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]/root/anaconda3/envs/llm/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 7/7 [02:01<00:00, 17.30s/it]


In [4]:
model = model.half().eval()

In [5]:
def chatglm(prompt, history=[], model=model, tokenizer=tokenizer):
    response, history = model.chat(tokenizer, 
                                   prompt , 
                                   history=[], 
                                   temperature=0.1, top_p=0.8, top_k=20,
                                   max_length=8192)
    return response

In [6]:
chatglm("你好")

'你好！很高兴见到你，欢迎问我任何问题。'

In [8]:
def chatglm_stream(prompt, history=[], model=model, tokenizer=tokenizer):
    for data in model.stream_chat(tokenizer, 
                                   prompt , 
                                   history=[], 
                                   temperature=0.1, top_p=0.8, top_k=20,
                                   max_length=8192):
        yield data[0]

In [12]:
from IPython.display import clear_output
user_prompt = "你好, 请介绍下自己"

In [13]:
for res in chatglm_stream(user_prompt):
    clear_output(wait=True)
    print(res)

你好！我是 ChatGLM3-6B，是清华大学KEG实验室和智谱AI公司共同训练的语言模型。我的目标是通过回答用户提出的问题来帮助他们解决问题。由于我是一个计算机程序，所以我没有自我意识，也不能像人类一样感知世界。我只能通过分析我所学到的信息来回答问题。


### 封装到langchain LLM里
LangChain是一个开源框架，它通过提供一系列工具、套件和接口，使开发者能够使用语言模型来实现各种复杂的任务，如文本到图像的生成、文档问答、聊天机器人等。LangChain简化了LLM应用程序生命周期的各个阶段，包括开发、生产化和部署。

LangChain具有六大核心组件，这些组件相互协作，形成一个强大而灵活的系统：

1. **模型（Models）**：包含各大语言模型的LangChain接口和调用细节，以及输出解析机制。
2. **提示模板（Prompts）**：使提示工程流线化，进一步激发大语言模型的潜力。
3. **数据检索（Indexes）**：构建并操作文档的方法，接受用户的查询并返回最相关的文档，轻松搭建本地知识库。
4. **记忆（Memory）**：通过短时记忆和长时记忆，在对话过程中存储和检索数据，增强ChatGPT等聊天机器人的记忆能力。
5. **链（Chains）**：LangChain中的核心机制，以特定方式封装各种功能，并通过一系列的组合，自动而灵活地完成任务。
6. **代理（Agents）**：通过“代理”让大模型自主调用外部工具和内部工具，使智能Agent成为可能。

In [14]:
from langchain.llms.base import LLM

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
from typing import Any, List, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
class ChatGLM(LLM):
    tokenizer : AutoTokenizer = None
    model: AutoModelForCausalLM = None

    def __init__(self, model, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.model = model
    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
        response, history = self.model.chat(self.tokenizer, prompt , history=[], temperature=0.1, top_p=0.8)
        return response
    @property
    def _llm_type(self) -> str:
        return "chatglm"

In [19]:
chatglm = ChatGLM(model, tokenizer)

In [20]:
chatglm("你好")

/tmp/ipykernel_30461/1897613078.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  chatglm("你好")


'你好！很高兴见到你，欢迎问我任何问题。'

## 6.通过Http API在线调用讯飞星火spark大模型

星火大模型API当前有Lite、Pro、Pro-128K、Max和4.0 Ultra五个版本，各版本独立计量tokens
``` shell

# python
pip install --upgrade spark_ai_python

# http
pip install requets
```
- https://www.xfyun.cn/doc/spark/HTTP%E8%B0%83%E7%94%A8%E6%96%87%E6%A1%A3.html

- **参数**

![](./data/sparkai.png)

- **key**

**https://console.xfyun.cn/services/bm35https://console.xfyun.cn/services/bm35**

![](./data/sparkkey.png)


### python 调用

In [23]:
import os
from sparkai.llm.llm import ChatSparkLLM, ChunkPrintHandler
from sparkai.core.messages import ChatMessage
from config import api_secret, api_key

appid = "5541c544"     #填写控制台中获取的 APPID 信息

domain = "generalv3"   # v1.5版本
spark_url = "ws://spark-api.xf-yun.com/v3.1/chat"  # v2.0环境的地址


In [24]:
from sparkai.log.logger import logger
from sparkai.core.callbacks import StdOutCallbackHandler
spark = ChatSparkLLM(
    spark_api_url=spark_url,
    spark_app_id=appid,
    spark_api_key=api_key,
    spark_api_secret=api_secret,
    spark_llm_domain=domain,
    streaming=False,
    max_tokens= 1024,
)

In [25]:
prompt = '你是谁？'

messages = [ChatMessage(
        role="user",
        content=prompt
    )]


history_msg = []
history = []
if len(history) != 0 :
    history_msg = [ChatMessage(role=msg[0], content=msg[1]) for msg in history]
    messages = history_msg + [ChatMessage(role="user", content=messages[0]['content'])]

handler = ChunkPrintHandler()
a = spark.generate([messages], callbacks=[handler])
print(a.generations[0][0].text)

您好，我是科大讯飞研发的认知智能大模型，我的名字叫讯飞星火认知大模型。我可以和人类进行自然交流，解答问题，高效完成各领域认知智能需求。


### spark http

In [28]:
import requests
import json
from IPython.display import clear_output
from config import http_key

url = "https://spark-api-open.xf-yun.com/v1/chat/completions"
data = {
        "model": "generalv3", # 指定请求的模型
        "messages": [
            {
                "role": "user",
                "content": "你是谁"
            }
        ],
        "stream": True
    }
header = {"Authorization": f"Bearer {http_key}"}
response = requests.post(url, headers=header, json=data, stream=True)

In [29]:
response.encoding = "utf-8"
info = ""
for line in response.iter_lines(decode_unicode="utf-8"):
    if 'data' in line:
        if 'DONE' in line:
            continue
        res = line.strip().replace('data: ', '')
        data = json.loads(res)
        cur_info = data['choices'][0]['delta']['content']
        print(cur_info, end='')

您好，我是科大讯飞研发的认知智能大模型，我的名字叫讯飞星火认知大模型。我可以和人类进行自然交流，解答问题，高效完成各领域认知智能需求。

### openai http

In [31]:
from openai import OpenAI
client = OpenAI(
        api_key=f"{http_key}", # APIPassword
        base_url = 'https://spark-api-open.xf-yun.com/v1' # 指向讯飞星火的请求地址
    )
response = client.chat.completions.create(
    model='generalv3', # 指定请求的版本
    messages = [
        {"role": "system", "content": "你是一个乐于助人的助手。"},
        {"role": "user", "content": "介绍下你自己"}
    ],
    temperature=0.7,  # 控制生成文本的随机性。越低越确定，越高越随机。
    top_p=0.9,       # 核采样 (Nucleus sampling)。top_p=0.9表示只考虑概率质量前90%的词。
    max_tokens=4096,  # 最大生成的token数量。
    stream=True      # 开启流式输出
)
print("生成的文本：")

for chunk in response:
    print(chunk.choices[0].delta.content, end='', flush=True)

生成的文本：
您好，作为一个认知智能模型。我的主要功能是提供信息查询、日程管理、智能推荐等服务。无论您需要解答哪种问题，或者需要完成哪种任务，只要告诉我，我都会尽我所能帮助您。同时，我还在不断学习和进步中，希望能更好地为您服务。

## 7.ollama(cpu/gpu)

Ollama是一个集成了多种大型语言模型的工具，它支持模型的部署、运行以及API的整合和调用

- 安装Ollama：
```shell
curl -fsSL https://ollama.com/install.sh | sh
```

- 验证安装：
```shell
# 输入来验证安装是否成功。
ollama --version
```

- 使用

``` shell
# 启动服务
ollama serve

# 运行模型

ollama run qwen2:70b

```



In [1]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='qwen2:72b',
)
chat_completion = client.chat.completions.create(
    messages=[
        {
            'role': 'user',
            'content': '介绍下你自己？',
        }
    ],
    max_tokens=4096,  # 最大生成的token数量。
    stream=True,      # 开启流式输出
    model='qwen2:72b',
    temperature=0.7,  # 控制生成文本的随机性。越低越确定，越高越随机。
    top_p=0.9,
)
for chunk in chat_completion:
    print(chunk.choices[0].delta.content, end='', flush=True)

我是来自阿里云的大规模语言模型，我叫通义千问。我是阿里云自主研发的超大规模语言模型，也能够生成与给定词语相关的同义词或短语，帮助用户丰富表达和拓宽思路。如果您有任何问题或需要帮助，请随时告诉我，我会尽力提供支持。